In [1]:
from citrination_client import CitrinationClient
from os import environ
from citrination_client import PifQuery, SystemQuery, PropertyQuery, ChemicalFieldQuery, FieldQuery, Filter
client = CitrinationClient(environ['CITRINATION_API_KEY'], 'https://citrination.com')

# Experimental design with Citrination

In this notebook, we demonstrate how to use Citrination to select experiments (or simulations, or literature searches) to maximize the impact on an optimization problem.

## Case study: thermoelectrics

Given a list of thermoelectric candidates, how many experiments do we need to find the best one?

Here, _best_ will be with respect to the thermoelecric figure of merit, ZT:
    $$ ZT = \frac{\sigma S^2 T}{\lambda}, $$
where $\sigma$ is the electrical conductivity, $S$ is the Seebeck coefficient, $T$ is the temperature, and $\lambda$ is the thermal conductivity.  Higher is better.

# Getting the data

We begin by querying for the thermoelectric dataset, which is on Citrination [here](https://citrination.com/datasets/150888).  We want the formula and zT

In [2]:
system_query = SystemQuery(
    chemical_formula=ChemicalFieldQuery(
        extract_as="Chemical formula"
    ),
    properties=PropertyQuery(
        name=FieldQuery(
            filter=[Filter(equal="ZT")]
        ),
        value=FieldQuery(
            extract_as="ZT"
        )
    )
)
thermoelectric_query = PifQuery(include_datasets=[150888], system=system_query)

Let's run it and see what we get:

In [3]:
search_result = client.search(thermoelectric_query)
print("We found {} records".format(search_result.total_num_hits))
print([x.extracted for x in search_result.hits[0:2]])

We found 165 records
[{'Chemical formula': 'La0.98Sr0.02CoO3', 'ZT': '0.026234732'}, {'Chemical formula': 'Zr0.4Hf0.4Ti0.2NiSn', 'ZT': '0.030958179'}]


This dataset has all the ZT values already, so we want to drop most of them before trying to design an experiment:

In [4]:
from random import shuffle
full_data = [x.extracted.copy() for x in search_result.hits]
shuffle(full_data)
known_subset = full_data[:20]
unknown_subset = full_data[20:]
for x in unknown_subset: del x['ZT']

Our goal is to pick the best material to measaure next from `unknown_subset`.

# Training a model on known data

We train a model using the csv -> dataset -> data_view workflow described in the [modeling tutorial](https://github.com/CitrineInformatics/learn-citrination/blob/master/MLonCitrination.ipynb).

## Create a csv

The csv just needs a header with the property names.

In [5]:
def write_csv(name, rows):
    with open(name, "w") as f:
        f.write("formula, ZT\n")
        for row in rows:
            f.write("{Chemical formula:s}, {ZT:s}\n".format(**row))
write_csv('known_thermoelectrics.csv', known_subset)

The rest of the model building process is on the website:
 1. Go to the [csv to datasets](https://citrination.com/csv_to_datasets) page and upload `known_thermoelectrics.csv`
 1. Follow the guide to create a dataset with formula and ZT values
 1. Go to the [data views page](https://citrination.com/data_views) and click "Create new dataset"
 1. Search for the property name "ZT" and select the dataset you created before.  Advance with the "NEXT >" button in the top right corner
 1. Follow the guide to create a data view that has `formula` and an input and `ZT` as an output

# Apply the model to unknown data

First, let's use the trained model to make predictions via the API.  Change the `view_id` below to point to your view.

In [6]:
view_id = 80
inputs = [{"Chemical formula": "PbTe"},]
resp = client.predict(str(view_id), inputs)
prediction = resp['candidates'][0]['ZT']
print("We predict the ZT of {} (@ 300K) to be {} +/- {}".format(inputs[0]['Chemical formula'], *prediction))

We predict the ZT of PbTe (@ 300K) to be 0.1092344663 +/- 0.04134446022951313


## Maximum likelihood of improvement

This tutorial is about _experimental design_, so we need to pick a criterion for experimental selection.

There are many, but a straight forward and powerful one is "maximum likelihood of improvement", which is easy to compute if we assume the output distribution is normal.

In [7]:
from scipy.special import erf
from math import sqrt
def probability_improvement(mean, sigma, baseline):
    return 0.5 * (1.0 + erf((mean - baseline) / (sigma * sqrt(2.0))))

What is the baseline?  The largest value in the known data:

In [8]:
baseline_ZT = max(float(x['ZT']) for x in known_subset)
print("The highest ZT value in the known subset is {}".format(baseline_ZT))

The highest ZT value in the known subset is 0.485042171


Now let's screen the unknown materials for likelihood of improvement:

In [9]:
predictions = client.predict(str(view_id), unknown_subset)['candidates']
for p in predictions:
    p['LI'] = probability_improvement(float(p['ZT'][0]), float(p['ZT'][1]), baseline_ZT)

Pandas can help us look at the result.  The top values for "LI" are the ones we should try next.

In [10]:
import pandas as pd
df = pd.DataFrame(predictions, columns=['Chemical formula', 'ZT', 'LI'])
df['Chemical formula'] = df['Chemical formula'].map(lambda x: x[0])
df['ZT'] = df['ZT'].map(lambda x: "{:5.2f} +/- {:5.2f}".format(*x))
df['LI'] = df['LI'].map(lambda x: "{:5.3f}".format(x))

df.sort_values('LI', axis=0, ascending=False)

,Chemical formula,ZT,LI
3,In0.3Co4Sb12,0.13 +/- 0.11,0.001
0,Na0.02PbTe0.85Se0.15,0.11 +/- 0.04,0.000
100,Ca0.7Ho0.3MnO3,0.02 +/- 0.01,0.000
93,Ba8Ga16Ge30,0.10 +/- 0.09,0.000
94,In2O3,0.08 +/- 0.04,0.000
95,In0.1Co4Sb12,0.12 +/- 0.10,0.000
96,Sr0.61Ba0.39Nb2O6,0.02 +/- 0.03,0.000
97,ZrNiSn,0.11 +/- 0.05,0.000
98,Mg2Si0.994Bi0.006,0.01 +/- 0.03,0.000
99,Si0.7956Ge0.1989P0.0055,0.09 +/- 0.02,0.000
